In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/cluster/group/karies_2022/Simone/karies/karies-models/AAA_BYOL_test/BYOL/byol_pretrain")

from pathlib import Path
from byol_pretrain.BYOL import BYOL
from byol_pretrain.toy_model import Model
from byol_pretrain.utils import get_loaders_CIFAR10, get_loaders_STL10

# Toy model

In [ ]:
model = Model(3).to("cuda")

batch_size = 50
pretrain_epochs = 100
test = 100 # else None to use all the data
log_dir = "./logs/proj_test/cifar10/DELETE/"
device="cuda"
lin_evaluation_frequency=20


loader, val_loader = get_loaders_CIFAR10(batch_size, test=test)
img_size = (32, 32)

# loader, val_loader = get_loaders_STL10(batch_size, test=test)
# img_size = (96, 96)

byol = BYOL(
    model,
    loader,
    val_loader,
    pretrain_epochs,
    log_dir,
    input_dims=3,
    img_dims=img_size,
    hidden_features=4096,
    device=device,
    lin_evaluation_frequency=lin_evaluation_frequency,
)

byol.pretrain()

## MaskRCNN

In [ ]:
from karies.models import MaskRCNN
from karies.config import MaskRCNNConfig, ModelConfig, Task, ModelTypes
from byol_pretrain.BYOL_MaskRCNN import BYOL_MaskRCNN, MaskRCNNModelWrapper

base_config: ModelConfig = {
    "name": "PRETRAIN-SIMONE-80epoch-batch6",
    "task": Task.training,
    "optimizer": "adam",
    "learning_rate": 0.0001,
    "weight_decay": 0.0001,
    "batch_size": 8,
    "num_workers": 0,
    "device": "cuda",
    "path_model": "/cluster/group/karies_2022/Simone/karies/karies-models/AAA_BYOL_test/BYOL/saved_pretrained_models/maskrcnn-weekend-test-batch-6/",
    "load_model_name": "pretrain_80_epochs.pth",
    "num_epochs": 100,
    "image_shape": [768, 1024],
    "dataset": "dataset_test",
    "labels_json": "labels_caries.json",
    "histogram_eq": False,
    "visualization_frequency": 1,
    "augmentations": [],
    'fix_random_seed':42
}

config: MaskRCNNConfig = {
    **base_config,
    "model_type": ModelTypes.MaskRCNN,
    "classes": 5,
    "iou_threshold": 0.1,
    "confidence_threshold": 0.1,
    "model_args": {},
    "loss_weights": [1.0, 1.0, 1.0, 1.0, 1.0],
}

maskrcnn = MaskRCNN(config, load=False)
wrap = MaskRCNNModelWrapper(
    maskrcnn.model.transform,
    maskrcnn.model.backbone,
)

loader, val_loader = maskrcnn.get_data_loaders()

In [ ]:
pretrain_epochs = 100
log_dir = Path("./logs/LOAD-DELETE")
device="cuda"
lin_evaluation_frequency=None

byol = BYOL_MaskRCNN(
    wrap,
    loader,
    val_loader,
    pretrain_epochs,
    log_dir,
    input_dims=3,
    img_dims=(700, 700),
    hidden_features=2048,
    device=device,
    lin_evaluation_frequency=lin_evaluation_frequency,
    mixed_precision_enabled=True,

    original_maskrcnn=maskrcnn,
    save_weights_every=5
)


byol.pretrain()